## Step1 : Create Store Class 


In [45]:
from sequence.kernel.timeline import Timeline

class Store(object):
    def __init__(self, tl: Timeline):
        self.opening = False
        self.timeline = tl

    def open(self) -> None:
        self.opening = True

    def close(self) -> None:
        self.opening = False

## Step 2: Schedule Events on Store

In [46]:
from sequence.kernel.event import Event
from sequence.kernel.process import Process

tl = Timeline() # create timeline
tl.show_progress = False # turn of progress bar, we will address this in later tutorials.
store = Store(tl) # create store

# open store at 7:00
open_proc = Process(store, 'open', []) # Process(object, function name: string, arguments of function: List[])
open_event = Event(7, open_proc) # Event(occurring time: int, process: Process)
tl.schedule(open_event) # Timeline.schedule(Event)

## In the above code, we first create instances of Timeline and Store. Then, we create a process with the object being Store, the method of Store and a list of arguments for the method. The Event class uses open_proc process to create an event occurring at 7:00. The object Timeline then schedules the created event in its event list.

In [47]:
print(store.opening) # False

False


## The shop is always closed until the scheduled event is executed. For the event to run and the shop to open, the timeline simulator must advance to time 7.

## Step 3: Run Simulation

In [48]:
tl.run()
print(tl.now(), store.opening) # 7 True

7 True


## Now, the state of store is updated to the open state. Meanwhile, the function Timeline.now() presents the current simulation time (7). When the simulator executes an event, the simulator will update its time to the timestamp of the event.

**Quiz**: can you change the store to close at 19:00?

In [49]:
close_proc = Process(store, 'close', [])
close_event = Event(19, close_proc)
tl.schedule(close_event)
tl.run()
print(tl.time, store.opening) # 19 False

19 False


**Question**: will the state of store change if we schedule close_event first, then open_event?

In [50]:
tl.time = 0
tl.schedule(open_event)
tl.schedule(close_event)
tl.run()
print(tl.time, store.opening)

19 False


In [51]:
tl.time = 0
tl.schedule(close_event)
tl.schedule(open_event)
tl.run()
print(tl.time, store.opening)

19 False


You can compare the results of the above programs. Both generate the same result (19 False), as the order of executing events does not rely on the order of calling the scheduling method

## Example: Automatically Open and Close Store
### Step 1: Upgrade Store Class
The previous example allows us to open and close the store manually. In this example, we upgrade Store to open and close automatically with 12 business hours. The code of the upgraded Store is shown below

In [52]:
from sequence.kernel.timeline import Timeline
from sequence.kernel.event import Event
from sequence.kernel.process import Process

class Store(object):
    def __init__(self, tl: Timeline):
        self.opening = False
        self.timeline = tl

    def open(self) -> None:
        self.opening = True
        process = Process(self, 'close', [])
        event = Event(self.timeline.now() + 12, process)
        self.timeline.schedule(event)

    def close(self) -> None:
        self.opening = False
        process = Process(self, 'open', [])
        event = Event(self.timeline.now() + 12, process)
        self.timeline.schedule(event)

The updated Store.open()(Store.close()) function schedules a Store.close() (Store.open()) after 12 hours. Now, the Store can repeatedly open and close every 12 hours.

### Step 2: Schedule Initial Event
We can then define a store with an initial state - that the store opens at 7.

In [53]:
tl = Timeline()
tl.show_progress = False
store = Store(tl)
process = Process(store, 'open', [])
event = Event(7, process)
tl.schedule(event)

However, if you use tl.run() to run the simulation, the simulation will get stuck in an infinite loop.

### Step3 : Define the Condition of Stopping Simulation and Run Simulation
There are two methods to terminate simulation:

set the stop time of simulation when creating the Timeline

stop the simulation in a function of Store

For the first method, we can construct the object of Timeline with the code below:

In [54]:
tl = Timeline(60) # simulate system for 60 hours


For the second method, we can call Timeline.stop() in the Store.open() and Store.close() methods:



In [55]:
from sequence.kernel.timeline import Timeline
from sequence.kernel.event import Event
from sequence.kernel.process import Process

class Store(object):
    def __init__(self, tl: Timeline):
        self.opening = False
        self.timeline = tl
    def open(self) -> None:
        if self.timeline.now() >= 60:
            self.timeline.stop()
        self.opening = True
        process = Process(self, 'close', [])
        event = Event(self.timeline.now() + 12, process)
        self.timeline.schedule(event)
    def close(self) -> None:
        if self.timeline.now() >= 60:
            self.timeline.stop()
        self.opening = False
        process = Process(self, 'open', [])
        event = Event(self.timeline.now() + 12, process)
        self.timeline.schedule(event)


with the above methods, we can observe the state of store after a specfic time.



In [56]:
for t in [15, 32, 52]:
    tl = Timeline(t)
    store = Store(tl)
    print(tl.now())
    
    process = Process(store, 'open', [])
    event = Event(7, process)
    tl.schedule(event)
    
    tl.run()
    print(store.opening)
# True, True, Fals

0
True
0
True
0
False
